In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial,name='weights')

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial,name='biases')
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [ ]:
tf.reset_default_graph()
def model(x,keep_prob):
    # layer 1
    with tf.variable_scope('conv1'):
        W_conv1 = weight_variable([5, 5, 1, 32])
        b_conv1 = bias_variable([32])
        x_image = tf.reshape(x, [-1,28,28,1])
        h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
        h_pool1 = max_pool_2x2(h_conv1)

    # layer 2
    with tf.variable_scope('conv2'):
        W_conv2 = weight_variable([5, 5, 32, 64])
        b_conv2 = bias_variable([64])
        h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
        h_pool2 = max_pool_2x2(h_conv2)
    
    with tf.variable_scope('fc1'):
        W_fc1 = weight_variable([7 * 7 * 64, 1024])
        b_fc1 = bias_variable([1024])
        h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    

    with tf.variable_scope('dropout'):
#        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
        h_fc1_drop = h_fc1
        
    with tf.variable_scope('fc2'):
        W_fc2 = weight_variable([1024, 10])
        b_fc2 = bias_variable([10])
        y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
    
    return y_conv

x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])        
keep_prob = tf.placeholder(tf.float32)

y_conv = model(x,keep_prob)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [ ]:
for i in tf.trainable_variables():
    print(i.name,i.get_shape())

In [ ]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    for i in range(20000):
        batch = mnist.train.next_batch(50)
        if i%100 == 0:
            train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1]})
            test_accuracy = accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})
            print("step %d, \t training accuracy %g\t test accuracy %g"%(i, train_accuracy,test_accuracy))
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})

# train model and save

In [ ]:
import pickle
output = open('mnist_lenet.pkl', 'wb')
pickle.dump(acc_list,'mnist_lenet.pkl')
output.close()

# old model

In [ ]:
from tensorflow.contrib import layers
def max_pool_2x2(tensor_in,name='maxpool'):
    with tf.variable_scope(name):
        out = tf.nn.max_pool(tensor_in,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    return out

def logit_accuracy(logit,one_hot):
    with tf.variable_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(logit, 1), tf.argmax(one_hot,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

def flatten(feat):
    _shape = feat.get_shape()
    nfeat = np.prod(_shape[1:]).value 
    with tf.variable_scope('flatten'):
        feat = tf.reshape(feat,shape=[-1,nfeat],name='flattened')
    return feat

def cross_entropy_loss(logit,onehot):
    with tf.variable_scope('cross_entropy'):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logit,onehot))
    return loss

def conv_model(x, onehot_tgt):
    x_image = tf.reshape(x, [-1,28,28,1])
    with tf.variable_scope('conv_layer1'):
        h_conv1 = layers.conv2d(x_image,32,[5,5],activation_fn=tf.nn.relu)
        h_pool1 = max_pool_2x2(h_conv1)
    
    with tf.variable_scope('conv_layer2'):
        h_conv2 = layers.conv2d(h_pool1,64,[5,5],activation_fn=tf.nn.relu)
        h_pool2 = max_pool_2x2(h_conv2)
    
    h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
    with tf.variable_scope('fc_layer1'):
        h_fc1 = layers.fully_connected(h_pool2_flat,1024,activation_fn=tf.nn.relu)

    with tf.variable_scope('fc_layer2'):
        logit = layers.fully_connected(h_fc1,10)

    tf.add_to_collection('logit',logit)
    
    accuracy = logit_accuracy(logit,onehot_tgt)
    tf.add_to_collection('accuracy',accuracy)
    
    loss = cross_entropy_loss(logit,onehot_tgt)
    tf.add_to_collection('loss',loss)
    
    train_op = layers.optimize_loss(loss,tf.contrib.framework.get_global_step(),optimizer='SGD',learning_rate=0.01)
    #train_op = tf.train.AdamOptimizer(1e-4).minimize(loss)

    tf.add_to_collection('train_op',train_op)

    return accuracy,loss,train_op

tf.reset_default_graph()

# build the model
x = tf.placeholder(shape=[None,784],dtype = tf.float32)
target = tf.placeholder(shape=[None,10],dtype = tf.int32)
acc,lo,train_op = conv_model(x,target)

In [ ]:
for i in tf.trainable_variables():
    print(i.name,i.get_shape())

In [ ]:
# saver
#saver = tf.train.Saver(max_to_keep=None)
# train the model
acc_list=[]
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    for i in xrange(20000):
        batch = mnist.train.next_batch(50)
        sess.run([train_op],feed_dict = {x:batch[0],target:batch[1]})
        if i%100==0:
            train_accuracy = acc.eval(feed_dict={x:batch[0], target: batch[1]})
            test_accuracy = acc.eval(feed_dict={x:mnist.test.images, target: mnist.test.labels})
            print("step %d, \t training accuracy %g\t test accuracy %g"%(i, train_accuracy,test_accuracy))
#            _acc = sess.run(acc,feed_dict = {x:mnist.test.images,target:mnist.test.labels})
#            save_path = saver.save(sess, "./mnist_lenet/model",global_step=i)
#            print("Model saved in file: %s" % save_path)
#            print(_acc)
#            acc_list.append(_acc)